In [1]:
#1 좌우로 구분
#SimpleITK,Nibabel,Numpy,Scipy 그리고 Scikit-image 등의 Python 라이브러리를 이용하여 늑골 이미지를 처리하고 분석하는 작업을 수행한다.
import SimpleITK as sitk
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops
from skimage.morphology import binary_closing

def load_nii_image(file_path): #Nifti 이미지를 불러와서 Numpy 배열로 변환하고 이미지 픽셀 간격(spacing)정보를 함께 반환
    nii_image = sitk.ReadImage(file_path)
    return sitk.GetArrayFromImage(nii_image), nii_image.GetSpacing()

def save_nii_image(image_array, spacing, file_path):# 주어진 Numpy 배열과 픽셀 간격 정보를 이용하여 Nifti 이미지를 생성하고 주어진 파일 경로에 저장
    nii_image = sitk.GetImageFromArray(image_array)
    nii_image.SetSpacing(spacing)
    sitk.WriteImage(nii_image, file_path)

def split_ribcage(ribcage_image, ribcage_spacing): #주어진 늑골 이미지를 좌우로 분할하여 왼쪽 늑골 이미지와 오른쪽 늑골 이미지를 생성
    left_ribcage = np.zeros_like(ribcage_image)
    right_ribcage = np.zeros_like(ribcage_image)
    mid_x = ribcage_image.shape[2] // 2
    left_ribcage[:, :, :mid_x] = ribcage_image[:, :, :mid_x]
    right_ribcage[:, :, mid_x:] = ribcage_image[:, :, mid_x:]
    return left_ribcage, right_ribcage

def process_single_rib(image_data, output_directory, affine): #주어진 늑골 이미지를 이용하여 이진 이미지를 생성하고 이를 닫힘 연산을 통해 스무딩한 후에 라벨링을 진행 그리고 
                                                              #각 라벨에 해당하는 영역의 크기가 1000보다 큰 경우 해당 영역을 분리하여 별도의 늑골 이미지를 생성하고 이를 NIFTI 파일로 저장
    threshold = 0.5
    binary_image = image_data > threshold

    closed_image = binary_closing(binary_image)
    labeled_image = label(closed_image)

    os.makedirs(output_directory, exist_ok=True)

    for region in regionprops(labeled_image):
        if region.area > 250:
            single_rib_image = np.zeros_like(labeled_image)
            single_rib_image[labeled_image == region.label] = 1
            output_file_name = f'rib_{region.label}.nii.gz'
            output_path = os.path.join(output_directory, output_file_name)
            single_rib_nifti = nib.Nifti1Image(single_rib_image, affine)
            nib.save(single_rib_nifti, output_path)

if __name__ == "__main__":
    base_input_directory = "D:/rib_research_hs/ribseg276labeling/3d"
    base_output_directory = "D:/rib_research_hs/ribseg276labeling"

    for ribfrac_number in range(0, 672):
        input_file_path = f"{base_input_directory}/RibFrac{ribfrac_number}-image_pred.nii"
        if os.path.exists(input_file_path):
            ribcage_image, ribcage_spacing = load_nii_image(input_file_path)
            left_ribcage, right_ribcage = split_ribcage(ribcage_image, ribcage_spacing)

            ribfrac_directory = f"{base_output_directory}/Ribfrac{ribfrac_number}"
            os.makedirs(ribfrac_directory, exist_ok=True)

            left_output_file_path = f"{ribfrac_directory}/left_ribcage.nii"
            right_output_file_path = f"{ribfrac_directory}/right_ribcage.nii"

            save_nii_image(left_ribcage, ribcage_spacing, left_output_file_path)
            save_nii_image(right_ribcage, ribcage_spacing, right_output_file_path)

            nifti_image = nib.load(left_output_file_path)
            image_data = nifti_image.get_fdata()

            output_directory = f'{ribfrac_directory}/output_directory_left'
            process_single_rib(image_data, output_directory, nifti_image.affine)

            nifti_image = nib.load(right_output_file_path)
            image_data = nifti_image.get_fdata()

            output_directory = f'{ribfrac_directory}/output_directory_right'
            process_single_rib(image_data, output_directory, nifti_image.affine)

In [3]:
#왼쪽 갈비뼈 넘버링
import os
import cv2
import numpy as np
import nibabel as nib
from skimage.measure import label, regionprops

# Define the function to process files
def process_files(files, directory, side):
    for file in files:
        file_path = os.path.join(directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        labeled_image = label(image_data)
        for region in regionprops(labeled_image):
            if region.area > 200: 
                bones[file] = {"side": side, "coords": region.coords, "z_coordinates": set(int(coord[2]) for coord in region.coords)}

for i in range(0, 670):
    right_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/output_directory_right'

    if not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    right_files = [file for file in os.listdir(right_directory) if file.endswith('.nii.gz')]
    merged_data = None
 
    for file in right_files:
        file_path = os.path.join(right_directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        if merged_data is None:
            merged_data = np.zeros_like(image_data)

        merged_data = np.maximum(merged_data, image_data)

    bones = {}
    process_files(right_files, right_directory, 'R')

    sorted_bones = sorted(bones.items(), key=lambda x: (x[1]['side'], min(x[1]['coords'], key=lambda coord: coord[0])[2]), reverse=True)

    start = 1
    for file, bone_info in sorted_bones:
        if bone_info['side'] == 'R':
            bone_number = f"L{start}"
            print(f"{file} is assigned with number {bone_number}.")
            bone_info['number'] = bone_number
            start += 1

    output_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_left'
    os.makedirs(output_directory, exist_ok=True)

    for z in range(merged_data.shape[2] - 1, -1, -1): 
        slice_data = merged_data[:, :, z]
        normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)
        
        numbers_to_render = []
        for file, bone_info in bones.items():
            if z in bone_info['z_coordinates']:
                coords_on_slice = [coord for coord in bone_info['coords'] if coord[2] == z]
                top_coord = min(coords_on_slice, key=lambda coord: coord[0])
                numbers_to_render.append((bone_info['number'], top_coord))

        bgr_data = cv2.cvtColor(normalized_data, cv2.COLOR_GRAY2BGR)
        
        for number, coord in numbers_to_render:
            text_size = cv2.getTextSize(number, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
            temp_canvas = np.zeros((text_size[1], text_size[0], 3), dtype=np.uint8)
            cv2.putText(temp_canvas, number, (0, text_size[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            rotated_text = cv2.rotate(temp_canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)

            region_start_x = int(coord[0])
            region_start_y = int(coord[1])
            region_end_x = region_start_x + rotated_text.shape[0]
            region_end_y = region_start_y + rotated_text.shape[1]

            # Ensure that the region does not exceed the bgr_data dimensions
            region_end_x = min(region_end_x, bgr_data.shape[0])
            region_end_y = min(region_end_y, bgr_data.shape[1])

            region = bgr_data[region_start_x:region_end_x, region_start_y:region_end_y]

            # Ensure that the rotated text does not exceed the region dimensions
            rotated_text = rotated_text[:region.shape[0], :region.shape[1]]

            red_channel = rotated_text[:, :, 2]
            mask = red_channel > 0
            region[mask] = rotated_text[mask]
            bgr_data[region_start_x:region_end_x, region_start_y:region_end_y] = region

        rotated_bgr_data = cv2.rotate(bgr_data, cv2.ROTATE_90_CLOCKWISE)

        output_file_path = os.path.join(output_directory, f"slice_{merged_data.shape[2]-z:03d}.png")
        cv2.imwrite(output_file_path, rotated_bgr_data)

print("The images have been sliced and saved.")


Skipping Ribfrac0 due to missing directory
Skipping Ribfrac1 due to missing directory
Skipping Ribfrac2 due to missing directory
Skipping Ribfrac3 due to missing directory
Skipping Ribfrac4 due to missing directory
Skipping Ribfrac5 due to missing directory
Skipping Ribfrac6 due to missing directory
Skipping Ribfrac7 due to missing directory
Skipping Ribfrac8 due to missing directory
Skipping Ribfrac9 due to missing directory
Skipping Ribfrac10 due to missing directory
Skipping Ribfrac11 due to missing directory
Skipping Ribfrac12 due to missing directory
Skipping Ribfrac13 due to missing directory
Skipping Ribfrac14 due to missing directory
Skipping Ribfrac15 due to missing directory
Skipping Ribfrac16 due to missing directory
Skipping Ribfrac17 due to missing directory
Skipping Ribfrac18 due to missing directory
Skipping Ribfrac19 due to missing directory
Skipping Ribfrac20 due to missing directory
Skipping Ribfrac21 due to missing directory
Skipping Ribfrac22 due to missing director

C:\Users\HS\AppData\Local\Temp\ipykernel_11652\904351078.py:60: RuntimeWarning: invalid value encountered in divide
  normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)
C:\Users\HS\AppData\Local\Temp\ipykernel_11652\904351078.py:60: RuntimeWarning: invalid value encountered in cast
  normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)


Skipping Ribfrac174 due to missing directory
Skipping Ribfrac175 due to missing directory
Skipping Ribfrac176 due to missing directory
Skipping Ribfrac177 due to missing directory
Skipping Ribfrac178 due to missing directory
Skipping Ribfrac179 due to missing directory
Skipping Ribfrac180 due to missing directory
Skipping Ribfrac181 due to missing directory
Skipping Ribfrac182 due to missing directory
Skipping Ribfrac183 due to missing directory
Skipping Ribfrac184 due to missing directory
Skipping Ribfrac185 due to missing directory
Skipping Ribfrac186 due to missing directory
Skipping Ribfrac187 due to missing directory
Skipping Ribfrac188 due to missing directory
Skipping Ribfrac189 due to missing directory
Skipping Ribfrac190 due to missing directory
Skipping Ribfrac191 due to missing directory
Skipping Ribfrac192 due to missing directory
Skipping Ribfrac193 due to missing directory
Skipping Ribfrac194 due to missing directory
Skipping Ribfrac195 due to missing directory
Skipping R

In [4]:
#오른쪽 갈비벼 넘버링
import os
import cv2
import numpy as np
import nibabel as nib
from skimage.measure import label, regionprops

# Define the function to process files
def process_files(files, directory, side):
    for file in files:
        file_path = os.path.join(directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        labeled_image = label(image_data)
        for region in regionprops(labeled_image):
            if region.area > 200: 
                bones[file] = {"side": side, "coords": region.coords, "z_coordinates": set(int(coord[2]) for coord in region.coords)}

for i in range(0, 670):
    right_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/output_directory_left'

    if not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    right_files = [file for file in os.listdir(right_directory) if file.endswith('.nii.gz')]
    merged_data = None

    for file in right_files:
        file_path = os.path.join(right_directory, file)
        nifti_image = nib.load(file_path)
        image_data = nifti_image.get_fdata()

        if merged_data is None:
            merged_data = np.zeros_like(image_data)

        merged_data = np.maximum(merged_data, image_data)

    bones = {}
    process_files(right_files, right_directory, 'R')

    sorted_bones = sorted(bones.items(), key=lambda x: (x[1]['side'], min(x[1]['coords'], key=lambda coord: coord[0])[2]), reverse=True)

    start = 1
    for file, bone_info in sorted_bones:
        if bone_info['side'] == 'R':
            bone_number = f"R{start}"
            print(f"{file} is assigned with number {bone_number}.")
            bone_info['number'] = bone_number
            start += 1

    output_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_right'
    os.makedirs(output_directory, exist_ok=True)

    for z in range(merged_data.shape[2] - 1, -1, -1): 
        slice_data = merged_data[:, :, z]
        normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)
        
        numbers_to_render = []
        for file, bone_info in bones.items():
            if z in bone_info['z_coordinates']:
                coords_on_slice = [coord for coord in bone_info['coords'] if coord[2] == z]
                top_coord = min(coords_on_slice, key=lambda coord: coord[0])
                numbers_to_render.append((bone_info['number'], top_coord))

        bgr_data = cv2.cvtColor(normalized_data, cv2.COLOR_GRAY2BGR)
        
        for number, coord in numbers_to_render:
            text_size = cv2.getTextSize(number, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
            temp_canvas = np.zeros((text_size[1], text_size[0], 3), dtype=np.uint8)
            cv2.putText(temp_canvas, number, (0, text_size[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            rotated_text = cv2.rotate(temp_canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)

            region_start_x = int(coord[0])
            region_start_y = int(coord[1])
            region_end_x = region_start_x + rotated_text.shape[0]
            region_end_y = region_start_y + rotated_text.shape[1]

            # Ensure that the region does not exceed the bgr_data dimensions
            region_end_x = min(region_end_x, bgr_data.shape[0])
            region_end_y = min(region_end_y, bgr_data.shape[1])

            region = bgr_data[region_start_x:region_end_x, region_start_y:region_end_y]

            # Ensure that the rotated text does not exceed the region dimensions
            rotated_text = rotated_text[:region.shape[0], :region.shape[1]]

            red_channel = rotated_text[:, :, 2]
            mask = red_channel > 0
            region[mask] = rotated_text[mask]
            bgr_data[region_start_x:region_end_x, region_start_y:region_end_y] = region

        rotated_bgr_data = cv2.rotate(bgr_data, cv2.ROTATE_90_CLOCKWISE)

        output_file_path = os.path.join(output_directory, f"slice_{merged_data.shape[2]-z:03d}.png")
        cv2.imwrite(output_file_path, rotated_bgr_data)

print("The images have been sliced and saved.")


Skipping Ribfrac0 due to missing directory
Skipping Ribfrac1 due to missing directory
Skipping Ribfrac2 due to missing directory
Skipping Ribfrac3 due to missing directory
Skipping Ribfrac4 due to missing directory
Skipping Ribfrac5 due to missing directory
Skipping Ribfrac6 due to missing directory
Skipping Ribfrac7 due to missing directory
Skipping Ribfrac8 due to missing directory
Skipping Ribfrac9 due to missing directory
Skipping Ribfrac10 due to missing directory
Skipping Ribfrac11 due to missing directory
Skipping Ribfrac12 due to missing directory
Skipping Ribfrac13 due to missing directory
Skipping Ribfrac14 due to missing directory
Skipping Ribfrac15 due to missing directory
Skipping Ribfrac16 due to missing directory
Skipping Ribfrac17 due to missing directory
Skipping Ribfrac18 due to missing directory
Skipping Ribfrac19 due to missing directory
Skipping Ribfrac20 due to missing directory
Skipping Ribfrac21 due to missing directory
Skipping Ribfrac22 due to missing director

C:\Users\HS\AppData\Local\Temp\ipykernel_11652\1148214002.py:60: RuntimeWarning: invalid value encountered in divide
  normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)
C:\Users\HS\AppData\Local\Temp\ipykernel_11652\1148214002.py:60: RuntimeWarning: invalid value encountered in cast
  normalized_data = ((slice_data / np.max(slice_data)) * 255).astype(np.uint8)


Skipping Ribfrac174 due to missing directory
Skipping Ribfrac175 due to missing directory
Skipping Ribfrac176 due to missing directory
Skipping Ribfrac177 due to missing directory
Skipping Ribfrac178 due to missing directory
Skipping Ribfrac179 due to missing directory
Skipping Ribfrac180 due to missing directory
Skipping Ribfrac181 due to missing directory
Skipping Ribfrac182 due to missing directory
Skipping Ribfrac183 due to missing directory
Skipping Ribfrac184 due to missing directory
Skipping Ribfrac185 due to missing directory
Skipping Ribfrac186 due to missing directory
Skipping Ribfrac187 due to missing directory
Skipping Ribfrac188 due to missing directory
Skipping Ribfrac189 due to missing directory
Skipping Ribfrac190 due to missing directory
Skipping Ribfrac191 due to missing directory
Skipping Ribfrac192 due to missing directory
Skipping Ribfrac193 due to missing directory
Skipping Ribfrac194 due to missing directory
Skipping Ribfrac195 due to missing directory
Skipping R

In [5]:
import os

In [6]:
#Left ribcage Right rib cage combine
import cv2
import numpy as np

# Loop through all directories from Ribfrac541 to Ribfrac670
for i in range(0, 670):

    # Define the directories where the left and right images are stored
    left_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_left'
    right_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_right'

    # Check if directories exist
    if not os.path.isdir(left_directory) or not os.path.isdir(right_directory):
        print(f"Skipping Ribfrac{i} due to missing directory")
        continue

    print(f"Processing Ribfrac{i}")

    # Iterate through all slices
    for z in range(999):  # Assuming max number of slices is 999
        left_image_path = os.path.join(left_directory, f"slice_{z:03d}.png")
        right_image_path = os.path.join(right_directory, f"slice_{z:03d}.png")

        # Check if both images exist
        if not os.path.isfile(left_image_path) or not os.path.isfile(right_image_path):
            print(f"Skipping slice {z} due to missing image")
            continue

        # Load images
        left_image = cv2.imread(left_image_path)
        right_image = cv2.imread(right_image_path)

        # Merge images. Note: This assumes that the images are properly aligned!
        merged_image = cv2.addWeighted(left_image, 0.5, right_image, 0.5, 0)

        # Save the merged image
        output_directory = f'D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_merged'
        os.makedirs(output_directory, exist_ok=True)
        output_image_path = os.path.join(output_directory, f"slice_{z:03d}.png")
        cv2.imwrite(output_image_path, merged_image)

print("The images have been merged and saved.")


Skipping Ribfrac0 due to missing directory
Skipping Ribfrac1 due to missing directory
Skipping Ribfrac2 due to missing directory
Skipping Ribfrac3 due to missing directory
Skipping Ribfrac4 due to missing directory
Skipping Ribfrac5 due to missing directory
Skipping Ribfrac6 due to missing directory
Skipping Ribfrac7 due to missing directory
Skipping Ribfrac8 due to missing directory
Skipping Ribfrac9 due to missing directory
Skipping Ribfrac10 due to missing directory
Skipping Ribfrac11 due to missing directory
Skipping Ribfrac12 due to missing directory
Skipping Ribfrac13 due to missing directory
Skipping Ribfrac14 due to missing directory
Skipping Ribfrac15 due to missing directory
Skipping Ribfrac16 due to missing directory
Skipping Ribfrac17 due to missing directory
Skipping Ribfrac18 due to missing directory
Skipping Ribfrac19 due to missing directory
Skipping Ribfrac20 due to missing directory
Skipping Ribfrac21 due to missing directory
Skipping Ribfrac22 due to missing director

In [36]:
#Raw data 슬라이스 
import os
import cv2
import numpy as np
import nibabel as nib

# Hounsfield Units (HU) 범위 정의
HOUNSFIELD_MIN = -700
HOUNSFIELD_MAX = 2000

# Normalize image based on Hounsfield Units
def normalizeImageIntensityRange(img):
    img = np.clip(img, HOUNSFIELD_MIN, HOUNSFIELD_MAX)
    return ((img - HOUNSFIELD_MIN) / (HOUNSFIELD_MAX - HOUNSFIELD_MIN)) * 255

# 기본 디렉터리 정의
source_directory = "C:/RIB/Ribfrac/test"
target_directory = "C:/RIB/Ribfrac/test_2dslice-700"

# 해당 폴더에 있는 .nii 파일들의 리스트를 얻기
nii_files = [file for file in os.listdir(source_directory) if file.endswith('.nii')]

for nii_file in nii_files:
    # .nii 파일을 불러오기
    nii_path = os.path.join(source_directory, nii_file)
    nifti_image = nib.load(nii_path)
    image_data = nifti_image.get_fdata()

    # Hounsfield Units 범위로 이미지 데이터 정규화
    normalized_data = normalizeImageIntensityRange(image_data).astype(np.uint8)

    # RibFrac 번호를 얻어서 해당 번호의 폴더를 생성
    ribfrac_number = nii_file.split('-')[0]
    ribfrac_folder = os.path.join(target_directory, ribfrac_number)
    os.makedirs(ribfrac_folder, exist_ok=True)

    # 슬라이스 순서를 반대로 진행
    for z in range(normalized_data.shape[2]-1, -1, -1):
        slice_data = normalized_data[:, :, z]
        bgr_data = cv2.cvtColor(slice_data, cv2.COLOR_GRAY2BGR)

        # 90도 회전하기
        rotated_bgr_data = cv2.rotate(bgr_data, cv2.ROTATE_90_CLOCKWISE)

        # 슬라이스 이미지를 RibFrac 번호의 폴더에 저장
        output_file_path = os.path.join(ribfrac_folder, f"slice_{normalized_data.shape[2]-z:03d}.png")
        cv2.imwrite(output_file_path, rotated_bgr_data)

print("The images have been sliced and saved.")


The images have been sliced and saved.


In [8]:
#테스트 2d slice와 sequential labeling slice 이미지 오버레이
#Final output
import os
import cv2
import numpy as np

# Overlay 두 이미지를 합치는 함수
def overlay_images(background_path, overlay_path, output_path, alpha=0.5):
    # 이미지를 읽어옵니다
    background = cv2.imread(background_path)
    overlay = cv2.imread(overlay_path)

    # 이미지의 크기가 다르다면 overlay 이미지의 크기를 background 이미지 크기에 맞춰 조절합니다
    if background.shape != overlay.shape:
        overlay = cv2.resize(overlay, (background.shape[1], background.shape[0]))

    # 두 이미지를 합치기 위해 addWeighted 함수를 사용합니다
    combined = cv2.addWeighted(background, alpha, overlay, 1 - alpha, 0)
    
    # 합쳐진 이미지를 저장합니다
    cv2.imwrite(output_path, combined)

# 폴더 내의 모든 슬라이스 이미지에 대해 overlay 수행
for i in range(1, 671):  # RibFrac1부터 RibFrac670까지
    merged_dir = f"D:/rib_research_hs/ribseg276labeling/Ribfrac{i}/number_merged"
    test_2dslice_dir = f"C:/RIB/Ribfrac/test_2dslice_final700/Ribfrac{i}"
    output_dir = f"D:/rib_research_hs/ribseg276labeling/Ribfrac{i}_finaloutput"

    # 만약 Ribfrac 번호의 폴더가 sequential_labeling 디렉터리에 없다면 다음 번호로 넘어갑니다.
    if not os.path.exists(f"D:/rib_research_hs/ribseg276labeling/Ribfrac{i}"):
        continue

    os.makedirs(output_dir, exist_ok=True)

    # 각 디렉터리 내의 파일들을 가져옵니다
    merged_files = os.listdir(merged_dir)
    test_2dslice_files = os.listdir(test_2dslice_dir)

    # 두 디렉터리에 있는 모든 슬라이스 이미지에 대해 overlay를 수행
    for merged_file, test_2dslice_file in zip(merged_files, test_2dslice_files):
        merged_path = os.path.join(merged_dir, merged_file)
        test_2dslice_path = os.path.join(test_2dslice_dir, test_2dslice_file)

        output_path = os.path.join(output_dir, f"overlay_{merged_file}")
        overlay_images(merged_path, test_2dslice_path, output_path)

print("Overlay completed for all slices.")

Overlay completed for all slices.
